In [1]:
from datasets import DatasetDict, Dataset, ClassLabel, load_dataset
import pandas as pd
import glob
from sklearn.model_selection import train_test_split
from src.utils import map_category


In [2]:
data_path = "data/interim/part-*.json"
json_files = glob.glob(data_path)
stream_df = pd.concat([pd.read_json(file, lines=True) for file in json_files], ignore_index=True)

stream_df["label"] = stream_df["main_category"].apply(map_category)
stream_df["text"] = stream_df["title"] + "\n" + stream_df["summary"]
stream_df = stream_df[["text", "label"]]


In [3]:
stream_df["label"].value_counts()


label
cs          5458
math        2205
cond-mat     915
physics      822
astro-ph     802
quant-ph     599
eess         565
hep          527
stat         271
gr-qc        213
nucl         116
q-bio         98
econ          93
nlin          54
math-ph       52
q-fin         46
Name: count, dtype: int64

In [4]:
train_df, temp_df = train_test_split(
    stream_df, 
    test_size=0.3,
    stratify=stream_df["label"],
    random_state=42
)

val_df, test_df = train_test_split(
    temp_df,
    test_size=0.5,
    stratify=temp_df["label"],
    random_state=42
)

train_df["label"].value_counts()

label
cs          3821
math        1543
cond-mat     641
physics      575
astro-ph     561
quant-ph     419
eess         396
hep          369
stat         190
gr-qc        149
nucl          81
q-bio         69
econ          65
nlin          38
math-ph       36
q-fin         32
Name: count, dtype: int64

In [5]:
aux_data = load_dataset("real-jiakai/arxiver-with-category")

aux_data.set_format(type="pandas")
aux_df = aux_data["train"][:]
aux_df["label"] = aux_df["primary_category"].apply(map_category)
aux_df["title"] = aux_df["title"].str.replace("\n  ", " ")
aux_df["text"] = aux_df["title"] + "\n" + aux_df["abstract"]
aux_df = aux_df[["text", "label"]]


README.md:   0%|          | 0.00/2.54k [00:00<?, ?B/s]

c:\Users\GPham\miniforge3\envs\assignment-03\Lib\site-packages\huggingface_hub\file_download.py:144: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\GPham\.cache\huggingface\hub\datasets--real-jiakai--arxiver-with-category. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


papers.parquet:   0%|          | 0.00/1.44G [00:00<?, ?B/s]

Generating train split:   0%|          | 0/63357 [00:00<?, ? examples/s]

In [6]:
aux_df["label"].value_counts()


label
cs          26733
math         9611
cond-mat     4660
astro-ph     4453
physics      4163
quant-ph     2930
hep          2900
eess         2839
stat         1546
gr-qc        1187
q-bio         677
nucl          512
math-ph       365
econ          288
nlin          262
q-fin         231
Name: count, dtype: int64

In [7]:
train_counts = train_df["label"].value_counts()
aux_counts = aux_df["label"].value_counts()

TARGET_CLASS_SIZE = 500

samples_to_take = {}

for label in train_counts.index.union(aux_counts.index):
    train_count = train_counts.get(label, 0)
    aux_count = aux_counts.get(label, 0)
    
    desired = min(TARGET_CLASS_SIZE, train_count + aux_count)
    need = desired - train_count
    take = min(need, aux_count)
    samples_to_take[label] = max(take, 0)  # avoid negatives


In [8]:
samples_to_take

{'astro-ph': 0,
 'cond-mat': 0,
 'cs': 0,
 'econ': 288,
 'eess': 104,
 'gr-qc': 351,
 'hep': 131,
 'math': 0,
 'math-ph': 365,
 'nlin': 262,
 'nucl': 419,
 'physics': 0,
 'q-bio': 431,
 'q-fin': 231,
 'quant-ph': 81,
 'stat': 310}

In [9]:
sampled_aux = []

for label, take_n in samples_to_take.items():
    if take_n > 0:
        class_subset = aux_df[aux_df["label"] == label]
        sampled = class_subset.sample(n=take_n, random_state=42)
        sampled_aux.append(sampled)

aug_aux_df = pd.concat(sampled_aux, ignore_index=True)

aug_train_df = pd.concat([train_df, aug_aux_df], ignore_index=True)

aug_train_df["label"].value_counts()


label
cs          3821
math        1543
cond-mat     641
physics      575
astro-ph     561
quant-ph     500
gr-qc        500
hep          500
stat         500
eess         500
q-bio        500
nucl         500
math-ph      401
econ         353
nlin         300
q-fin        263
Name: count, dtype: int64

In [10]:
# sub_aux_df = aux_df.loc[aux_df["label"].isin(["econ", "nlin", "nucl-ex", "q-bio","hep-lat", "hep-ex", "math-ph", "nucl-th","q-fin", "hep-th"])]


In [11]:
# sub_aux_df["label"].value_counts()


In [12]:
# aug_train_df = pd.concat([train_df, sub_aux_df])
print(f"Shape after augmentation: {aug_train_df.shape}")
aug_train_df["label"].value_counts()


Shape after augmentation: (11958, 2)


label
cs          3821
math        1543
cond-mat     641
physics      575
astro-ph     561
quant-ph     500
gr-qc        500
hep          500
stat         500
eess         500
q-bio        500
nucl         500
math-ph      401
econ         353
nlin         300
q-fin        263
Name: count, dtype: int64

In [13]:
aug_stream_data = DatasetDict({
    "train": Dataset.from_pandas(aug_train_df, preserve_index=False),
    "validation": Dataset.from_pandas(val_df, preserve_index=False),
    "test": Dataset.from_pandas(test_df, preserve_index=False)
})

labels = sorted(aug_train_df["label"].unique())
class_label = ClassLabel(names=labels)

aug_stream_data = aug_stream_data.cast_column("label", class_label)

aug_stream_data.save_to_disk("data/processed/aug_stream_data")


Casting the dataset:   0%|          | 0/11958 [00:00<?, ? examples/s]

Casting the dataset:   0%|          | 0/1925 [00:00<?, ? examples/s]

Casting the dataset:   0%|          | 0/1926 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/11958 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/1925 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/1926 [00:00<?, ? examples/s]